In [70]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [71]:
# Loading the api keys from your local .env file
import algo.exchange_setup
client = algo.exchange_setup.establish_connection()

In [97]:
from binance import Client
import json
import datetime
import math
from algo.datamodel import *
from collections import deque

# TODO: import system time and figure out how to keep track of time based on different input time units

class MarketState:
    """ Data class that holds all the information of the market about a pair needed for our algo """

    def __init__(self, client: Client = None, window_size: int = 1000, time_unit: str = "seconds"):
        assert (client != None and window_size > 0)
        self.client = client
        self.window_size = window_size  # TODO: define smallest time unit - use seconds or nanoseconds?
        self.ticker_prices = {}  # key: symbol (str), value: price (float)
        self.time_unit = time_unit
        self.time = None  # TODO: implement

        if time_unit == "seconds":
            self.time_increment = 1 # increment in epoch seconds


        self.__fetch_prices()
        self.symbols = list(self.ticker_prices.keys())

        self.window = deque([]) # double-ended queue for O(1) pop from front and O(1) insert from back of queue
        self.__fill_window()


    def update(self):
        """
        Updates the MarketState by fetching the latest data from the exchange.
        This function should be called on every tick (as defined by the algorithm).
        Other functions in this class will rely on the internal states that this function updates,
        so this function should be called before any other functions in each time instance.
        """
        self.__fetch_prices()
        self.__update_window()

    def __fetch_prices(self):
        """Private function that updates the prices within the MarketState class"""
        # TODO: update time
        for t in self.client.get_all_tickers():
            symbol, price = t["symbol"], float(t["price"])
            self.ticker_prices[symbol] = price

    def __fill_window(self):
        """Fill the window with historical data"""
        pass

    def __update_window(self):
        """Update the window with the current period"""
        pass

    def current_price(self, coin: str) -> float:
        """Get the current price of a specific coin"""
        return self.ticker_prices[coin]

    def portfolio_balance(self) -> float:
        """Total USD value of all coins in exchange that we are holding"""
        # need to all get_asset_balance and multiply by market price of the coin-USDT pair
        # TODO: update balance inside the update function? or separately?
        balances = [Balance(**b) for b in self.client.get_account()["balances"]]
        total_balance = 0
        excluded_coins = []
        for b in balances:
            if b.asset in ("USDT", "BUSD"):
                # stablecoin values reflect real value of USD
                total_balance += b.free + b.locked
            else:
                # need to convert this cryptocurrency to USDT to get price in USD
                ticker_usdt = b.asset + 'USDT'  # remark: assume all pairs have a USDT conversion

                if not (ticker_usdt in self.ticker_prices):
                    # coins excluded from portfolio balance (no direct conversion to USD available)
                    # TODO: find a conversion from these coins to BTC to USDT (maybe try BUSD)
                    excluded_coins.append(b.asset)
                    continue

                qty = b.free + b.locked
                total_balance += qty * self.ticker_prices[ticker_usdt]

        print(
            f"Excluded the following coins from portfolio balance calulations (could not find conversion to USD): {excluded_coins}")

        return total_balance

    def hard_stop_loss(self) -> float:
        # this will move up/down as we our portfolio increases/decreases in value,
        #   do we want to set an absolute cutoff to limit exposure or are we fine with increased exposure we our portfolio grows?
        # TODO: change to John's new stop loss formula?
        return 0.005 * self.portfolio_balance()

    def derivative_of_spread(self) -> float:
        # return the derivative of the spread price (slope of the spread)
        # TODO: clarify formula: (change in price of spread) / (time or ??)
        # TODO: internal timer, 1 second for now, but make it adjustable
        pass

    def current_spread(self, coin1: str, coin2: str, beta: float) -> float:
        """
        Formula: spread = coin1 - coin2 * beta

        Example:
            Suppose that beta = 7.4
            Then let's say you want to buy this spread, and you want to do 10 shares of coin1
            Then you buy 10 shares of coin1, and you short 74 shares of coin2
        """
        # XXX: potential issue with fractional shares due to the exchange's price/qty filters (need to verify)
        return self.current_price(coin1) - self.current_price(coin2) * beta

    def spread_moving_avg(self, coin1, coin2) -> float:
        # Get Rolling Window Price Change Statistics
        # GET /api/v3/ticker
        pass

    def spread_upper_bollinger_band(self, coin1, coin2) -> float:
        pass

    def spread_lower_bollinger_band(self, coin1, coin2) -> float:
        pass


In [73]:
# Initializer
state = MarketState(client)

In [74]:
tickers = client.get_orderbook_tickers()

In [76]:
avg_price = client.get_avg_price(symbol='BNBBTC')

In [77]:
avg_price

{'mins': 1, 'price': '0.00716199'}

In [78]:
candles = client.get_klines(symbol='BNBBTC', interval=Client.KLINE_INTERVAL_30MINUTE)
candles

[[1697045400000,
  '0.00763200',
  '0.00763200',
  '0.00763200',
  '0.00763200',
  '0.00000000',
  1697047199999,
  '0.00000000',
  0,
  '0.00000000',
  '0.00000000',
  '0'],
 [1697047200000,
  '0.00763200',
  '0.00763200',
  '0.00763200',
  '0.00763200',
  '0.00000000',
  1697048999999,
  '0.00000000',
  0,
  '0.00000000',
  '0.00000000',
  '0'],
 [1697049000000,
  '0.00763200',
  '0.00763200',
  '0.00763200',
  '0.00763200',
  '0.00000000',
  1697050799999,
  '0.00000000',
  0,
  '0.00000000',
  '0.00000000',
  '0'],
 [1697050800000,
  '0.00763200',
  '0.00763200',
  '0.00763200',
  '0.00763200',
  '0.00000000',
  1697052599999,
  '0.00000000',
  0,
  '0.00000000',
  '0.00000000',
  '0'],
 [1697052600000,
  '0.00763100',
  '0.00763100',
  '0.00763100',
  '0.00763100',
  '0.58500000',
  1697054399999,
  '0.00446413',
  1,
  '0.00000000',
  '0.00000000',
  '0'],
 [1697054400000,
  '0.00763100',
  '0.00763100',
  '0.00763100',
  '0.00763100',
  '0.00000000',
  1697056199999,
  '0.000000

In [79]:
tickers = client.get_ticker()
tickers

[{'symbol': 'ETHBTC',
  'priceChange': '0.00044000',
  'priceChangePercent': '0.815',
  'weightedAvgPrice': '0.05427481',
  'prevClosePrice': '0.05400000',
  'lastPrice': '0.05445000',
  'lastQty': '0.10770000',
  'bidPrice': '0.05443000',
  'bidQty': '0.03310000',
  'askPrice': '0.05445000',
  'askQty': '0.15140000',
  'openPrice': '0.05401000',
  'highPrice': '0.05591000',
  'lowPrice': '0.05383000',
  'volume': '172.88770000',
  'quoteVolume': '9.38344630',
  'openTime': 1697858458290,
  'closeTime': 1697944858290,
  'firstId': 33939,
  'lastId': 35477,
  'count': 1539},
 {'symbol': 'LTCBTC',
  'priceChange': '0.00001300',
  'priceChangePercent': '0.602',
  'weightedAvgPrice': '0.00216225',
  'prevClosePrice': '0.00216100',
  'lastPrice': '0.00217400',
  'lastQty': '13.06400000',
  'bidPrice': '0.00217300',
  'bidQty': '0.96600000',
  'askPrice': '0.00217400',
  'askQty': '2.89800000',
  'openPrice': '0.00216100',
  'highPrice': '0.00218700',
  'lowPrice': '0.00214900',
  'volume': 

In [83]:
import time
import math

In [91]:
server_time = client.get_server_time()['serverTime']
client_time = time.time() * 10**3

In [93]:
print(server_time, math.log10(server_time))
print(client_time, math.log10(client_time))

1697946907108 12.22992410621976
1697946907185.456 12.229924106239572


In [84]:
math.log10(time.time())

9.229924079470619

In [94]:
time.ctime()

'Sat Oct 21 22:56:09 2023'

In [95]:
candles = client.get_klines(symbol='BNBBTC', interval=Client.KLINE_INTERVAL_30MINUTE)

In [99]:
first_kline = Kline(*candles[0])

In [100]:
first_kline

Kline(openTime=1697049000000, open='0.00763200', high='0.00763200', low='0.00763200', close='0.00763200', volume='0.00000000', closeTime=1697050799999, quoteAssetVol='0.00000000', numTrades=0, takerBuyBaseAssetVol='0.00000000', takerBuyQuoteAssetVol='0.00000000', __ignore__='0')

In [101]:
time.time_ns()

1697948594762721000